In [1]:
import openai
import os
from dotenv import load_dotenv
from pinecone import Pinecone,ServerlessSpec
load_dotenv()

pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])

In [2]:
# 인덱스 생성
index_name = "movie-index"
if not pc.has_index(index_name):
    pc.create_index(name=index_name,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
index = pc.Index(index_name)

c:\Users\USER\miniconda3\envs\api\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%pip install pandas numpy


  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 11.0/11.0 MB 78.8 MB/s eta 0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/3 [pytz]
   ------------- -------------------------- 1/3 [tzdata]
   ------------- -------------------------- 1/3 [tzdata]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------

In [4]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
# 메타데이터  (csv 로드)
data = [
    {
        "title": "응답하라 1988",
        "year": 2015,
        "genre": ["드라마", "코미디"],
        "director": "신원호",
        "actors": ["혜리", "박보검", "류준열"],
        "rating": 9.2,
        "synopsis": "1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마."
    },
    {
        "title": "기생충",
        "year": 2019,
        "genre": ["드라마", "스릴러"],
        "director": "봉준호",
        "actors": ["송강호", "이선균", "조여정"],
        "rating": 8.6,
        "synopsis": "가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화."
    }
]
import pandas as pd
df = pd.DataFrame(data)
texts = df['synopsis'].tolist()
import openai
response = openai.embeddings.create(
    input=texts,
    model = 'text-embedding-3-small'
)

In [13]:
embeddings = [item.embedding for item in response.data]
len(embeddings[0])

1536

In [14]:
# 업서트할 벡터 레코드 리스트 구성
records_to_upsert = []
for idx, row in df.iterrows():
    vec_id = f"movie-{idx}"  # 고유 ID 생성
    vec = embeddings[idx]    # 해당 텍스트의 임베딩 벡터
    # 메타데이터 구성 (DataFrame row를 딕셔너리로 변환)
    meta = {
        "title": row["title"],
        "year": int(row["year"]),
        "genre": row["genre"],        # 리스트 또는 문자열
        "director": row["director"],
        "actors": row["actors"],      # 리스트
        "rating": float(row["rating"]),
        "synopsis": row["synopsis"]   # 원문 텍스트 저장 (옵션)
    }
    records_to_upsert.append((vec_id, vec, meta))

# Pinecone 업서트 (한번에 여러 벡터 저장)
index.upsert(vectors=records_to_upsert)

{'upserted_count': 2}

In [15]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 2}},
 'total_vector_count': 2,
 'vector_type': 'dense'}